In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(celldex)
    library(ShinyCell)
    library(dplyr)
    library(SingleR)
    library(dplyr)
})

In [ ]:
counts <- readRDS("data/2023-04-05_rawCounts_Sc_QCed.rds")
metadata <- readRDS("data/2023-04-05_metaData_Sc_QCed.rds")

obj <- CreateSeuratObject(counts = counts, meta.data = metadata, min.cells = 3, min.features = 300)
obj
save(obj, file = "objects/00_raw.rda")

In [ ]:
# QC
obj[["percent.mt"]] <- PercentageFeatureSet(object = obj, pattern = "^MT-")
obj
save(obj, file = file.path("objects", "01.rda"))

In [ ]:
# Normalization
obj <- SCTransform(obj, ncells = 3000,  variable.features.n = 2000, vst.flavor="v2", method = 'glmGamPoi', conserve.memory = TRUE)
save(obj, file=file.path("objects", "02.rda"))

In [4]:
obj <- RunPCA(obj, npcs = 30, verbose = TRUE)
obj <- RunUMAP(obj, reduction = "pca", dims = 1:20)
obj <- FindNeighbors(obj, reduction = "pca", dims = 1:20)
obj <- FindClusters(obj, resolution = 0.5)
save(obj, file=file.path("objects", "03.rda"))

PC_ 1 
Positive:  ALDOB, MT1G, GPX3, MT1H, UMOD, MIOX, GATM, GSTA1, FABP1, PCK1 
	   UGT2B7, HPD, CUBN, MT1X, NAT8, PDZK1IP1, BHMT, LRP2, GSTA2, ASS1 
	   FXYD2, APOE, MT1F, AK4, SLC16A9, PAH, SLC5A12, ACY1, ACSM2B, AGMAT 
Negative:  LYZ, TYROBP, C1QA, C1QB, FCER1G, AIF1, HLA-DRA, S100A9, C1QC, HLA-DPA1 
	   TMSB4X, CTSS, S100A4, HLA-DPB1, LST1, FCN1, HLA-DRB1, S100A8, SRGN, IGFBP5 
	   RNASE1, MS4A7, CD74, FABP5, LGALS1, CD14, VIM, FCGR3A, IFI27, MGP 
PC_ 2 
Positive:  ALDOB, MT1G, GPX3, FTL, MT1H, MIOX, APOE, GATM, LYZ, TYROBP 
	   C1QA, C1QB, GSTA1, FABP1, UGT2B7, FCER1G, AIF1, HPD, CUBN, PDZK1IP1 
	   NAT8, MT1X, C1QC, PCK1, S100A9, LRP2, BHMT, HLA-DRA, ASS1, HLA-DPA1 
Negative:  UMOD, SLC12A1, DEFB1, KNG1, TMEM52B, WFDC2, SLC12A3, CALB1, AQP2, SFRP1 
	   S100A2, ATP1B1, ATP1A1, CLDN16, FXYD4, EGF, IGFBP7, SERPINA5, MALAT1, S100A6 
	   MAL, CKB, CLDN10, DUSP9, MUC1, CA12, KCNJ1, WNK1, SPINK1, HSD11B2 
PC_ 3 
Positive:  UMOD, LYZ, TYROBP, C1QA, C1QB, FCER1G, SLC12A1, AIF1, S100A9, C

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 547796
Number of edges: 17707089

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9534
Number of communities: 29
Elapsed time: 763 seconds


In [5]:
# Load annotation
monaco.ref <- celldex::MonacoImmuneData()
blueprint.ref <- celldex::BlueprintEncodeData()

see ?celldex and browseVignettes('celldex') for documentation

loading from cache

see ?celldex and browseVignettes('celldex') for documentation

loading from cache

see ?celldex and browseVignettes('celldex') for documentation

loading from cache

see ?celldex and browseVignettes('celldex') for documentation

loading from cache



In [6]:
# Blueprint annnotation
sce <- LayerData(obj)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
obj@meta.data$blueprint.main <- blueprint.main$pruned.labels
save(obj, file=file.path("objects", "04.rda"))

In [7]:
# Monaco annontation
sce <- LayerData(obj)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
obj@meta.data$monaco.fine <- monaco.fine$pruned.labels
save(obj, file=file.path('objects', '05.rda'))

In [8]:
# Update metadata
obj@meta.data$Stim <- obj@meta.data$Type
obj@meta.data$Tissue_Type <- obj@meta.data$broad.group
obj@meta.data$Fine_Cell_Type <- obj@meta.data$fine.type
obj@meta.data$Broad_Cell_Type <- obj@meta.data$broad.type
save(obj, file=file.path('objects', '06.rda'))

## Process Individual Cell Types

In [1]:
load(file=file.path("objects", "02.rda"))

In [ ]:
source("R/process.R")

In [ ]:
### Write shiny files

#bcell
seu =  get(load(file = "objects/BCellmonaco05.rda"))
colnames(seu@meta.data)[10] <- "monaco.fine"
scConf1 = createConfig(seu)
makeShinyFiles(seu, scConf1,
             gene.mapping = TRUE,
             shiny.prefix = "sc1",
             shiny.dir = "amp2_sle_scRNAseq_immune/",
             gex.assay = "SCT")

#myeloid
seu =  get(load(file = "objects/MyeloidCellmonaco05.rda"))
colnames(seu@meta.data)[10] <- "monaco.fine"
scConf2 = createConfig(seu)
makeShinyFiles(seu, scConf2,
             gene.mapping = TRUE,
             shiny.prefix = "sc2",
             shiny.dir = "amp2_sle_scRNAseq_immune/",
             gex.assay = "SCT")

#nk
seu =  get(load(file = "objects/NKCellmonaco05.rda"))
colnames(seu@meta.data)[10] <- "monaco.fine"
scConf3 = createConfig(seu)
makeShinyFiles(seu, scConf3,
             gene.mapping = TRUE,
             shiny.prefix = "sc3",
             shiny.dir = "amp2_sle_scRNAseq_immune/",
             gex.assay = "SCT")

#plasma
seu =  get(load(file = "objects/PlasmaCellmonaco05.rda"))
colnames(seu@meta.data)[10] <- "monaco.fine"
scConf4 = createConfig(seu)
makeShinyFiles(seu, scConf4,
             gene.mapping = TRUE,
             shiny.prefix = "sc4",
             shiny.dir = "amp2_sle_scRNAseq_immune/",
             gex.assay = "SCT")

#tcell
seu =  get(load(file = "objects/TCellmonaco05.rda"))
colnames(seu@meta.data)[10] <- "monaco.fine"
scConf5 = createConfig(seu)
makeShinyFiles(seu, scConf5,
             gene.mapping = TRUE,
             shiny.prefix = "sc5",
             shiny.dir = "amp2_sle_scRNAseq_immune/",
             gex.assay = "SCT")

# All
seu =  get(load(file = "objects/06.rda"))
seu <- SetIdent(seu, value = "Broad_Cell_Type")
cells <-  c('T Cell', 'NK Cell', 'Plasma Cell', 'B Cell', 'Myeloid Cell')
seu <- subset(seu, Broad_Cell_Type  %in%  cells)

# Rename
seu@meta.data$Stim <- seu@meta.data$Type
seu@meta.data$Tissue_Type <- seu@meta.data$broad.group
seu@meta.data$Fine_Cell_Type <- seu@meta.data$fine.type
seu@meta.data$Broad_Cell_Type <- seu@meta.data$broad.type

columns_to_keep <-  c('orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Stim', 'Tissue_Type',
 'Fine_Cell_Type', 'Broad_Cell_Type', 'monaco.fine')
seu@meta.data <- seu@meta.data[, columns_to_keep, drop = FALSE]

# Recluster
seu <- RunPCA(seu, npcs = 30, verbose = TRUE)
seu <- RunUMAP(seu, reduction = "pca", dims = 1:20)
seu <- FindNeighbors(seu, reduction = "pca", dims = 1:20)
seu <- FindClusters(seu, resolution = 0.5)

scConf6 = createConfig(seu, maxLevels = 60)
makeShinyFiles(seu, scConf6,
             gene.mapping = TRUE,
             shiny.prefix = "sc6",
             shiny.dir = "amp2_sle_scRNAseq_immune/",
             gex.assay = "SCT")

makeShinyCodesMulti(
  shiny.title = "AMP Phase II SLE, Immune Cells", shiny.footnotes = NULL,
  shiny.prefix = c("sc1", "sc2", "sc3", "sc4", "sc5", "sc6"),
  shiny.headers = c("B Cell", "Myeloid", "NK", "Plasma Cell", "T Cell", "All Cells"), 
  shiny.dir = "amp2_sle_scRNAseq_immune/") 

In [3]:
system("R -e \"shiny::runApp('amp2_sle_scRNAseq_immune')\"")

In [ ]:
# ### Write shiny files

# #dn
# seu =  get(load(file = "objects/DNblueprint05.rda"))
# colnames(seu@meta.data)[10] <- "blueprint.main"
# scConf1 = createConfig(seu)
# makeShinyFiles(seu, scConf1,
#              gene.mapping = TRUE,
#              shiny.prefix = "sc1",
#              shiny.dir = "amp2_sle_scRNAseq_kidney/",
#              gex.assay = "SCT")

# #glom
# seu =  get(load(file = "objects/GLOMblueprint05.rda"))
# colnames(seu@meta.data)[10] <- "blueprint.main"
# scConf2 = createConfig(seu)
# makeShinyFiles(seu, scConf2,
#              gene.mapping = TRUE,
#              shiny.prefix = "sc2",
#              shiny.dir = "amp2_sle_scRNAseq_kidney/",
#              gex.assay = "SCT")

# #intl
# seu =  get(load(file = "objects/INTLblueprint05.rda"))
# colnames(seu@meta.data)[10] <- "blueprint.main"
# scConf3 = createConfig(seu)
# makeShinyFiles(seu, scConf3,
#              gene.mapping = TRUE,
#              shiny.prefix = "sc3",
#              shiny.dir = "amp2_sle_scRNAseq_kidney/",
#              gex.assay = "SCT")

# #loh
# seu =  get(load(file = "objects/LOHblueprint05.rda"))
# colnames(seu@meta.data)[10] <- "blueprint.main"
# scConf4 = createConfig(seu)
# makeShinyFiles(seu, scConf4,
#              gene.mapping = TRUE,
#              shiny.prefix = "sc4",
#              shiny.dir = "amp2_sle_scRNAseq_kidney/",
#              gex.assay = "SCT")

# #und   
# seu =  get(load(file = "objects/UNDblueprint05.rda"))
# colnames(seu@meta.data)[10] <- "blueprint.main"
# scConf5 = createConfig(seu)
# makeShinyFiles(seu, scConf5,
#              gene.mapping = TRUE,
#              shiny.prefix = "sc11",
#              shiny.dir = "amp2_sle_scRNAseq_kidney/",
#              gex.assay = "SCT")

# #pt
# seu =  get(load(file = "objects/PTblueprint05.rda"))
# colnames(seu@meta.data)[10] <- "blueprint.main"
# scConf6 = createConfig(seu)
# makeShinyFiles(seu, scConf6,
#              gene.mapping = TRUE,
#              shiny.prefix = "sc6",
#              shiny.dir = "amp2_sle_scRNAseq_kidney/",
#              gex.assay = "SCT")

# All
seu =  get(load(file = "objects/06.rda"))
seu <- SetIdent(seu, value = "Broad_Cell_Type")
cells <-  c('PT', 'INTL', 'GLOM', 'DN', 'LOH', 'UND')
seu <- subset(seu, Broad_Cell_Type  %in%  cells)

# Rename
seu@meta.data$Stim <- seu@meta.data$Type
seu@meta.data$Tissue_Type <- seu@meta.data$broad.group
seu@meta.data$Fine_Cell_Type <- seu@meta.data$fine.type
seu@meta.data$Broad_Cell_Type <- seu@meta.data$broad.type

columns_to_keep <-  c('orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Stim', 'Tissue_Type',
 'Fine_Cell_Type', 'Broad_Cell_Type', 'blueprint.main')
seu@meta.data <- seu@meta.data[, columns_to_keep, drop = FALSE]

# Recluster
seu <- RunPCA(seu, npcs = 30, verbose = TRUE)
seu <- RunUMAP(seu, reduction = "pca", dims = 1:20)
seu <- FindNeighbors(seu, reduction = "pca", dims = 1:20)
seu <- FindClusters(seu, resolution = 0.5)

scConf7 = createConfig(seu)
makeShinyFiles(seu, scConf7,
             gene.mapping = TRUE,
             shiny.prefix = "sc7",
             shiny.dir = "amp2_sle_scRNAseq_kidney/",
             gex.assay = "SCT")

makeShinyCodesMulti(
  shiny.title = "AMP Phase II SLE, Structural Cells", shiny.footnotes = NULL,
  shiny.prefix = c("sc1", "sc2", "sc3", "sc4", "sc5", "sc6", "sc7"),
  shiny.headers = c("Distal Nephron (DN)", "Glomerulus (GLOM)", "Interstitial (INTL)", "Loop of Henle (LOH)",
   "Undetermined (UND)", "Proximal Tube (PT)", "All Cells"), 
  shiny.dir = "amp2_sle_scRNAseq_kidney/") 

In [4]:
system("R -e \"shiny::runApp('amp2_sle_scRNAseq_kidney')\"")